# OpenSSL Trial
This notebook serves to compare node-forge's AES with OpenSSL's AES implementation.

And maybe RSA, if we know how to test the implementation later on.

### Checking OpenSSL version
First we check and see if Colab's Linux environment has OpenSSL installed

In [ ]:
!openssl version -a

OpenSSL 1.1.1  11 Sep 2018
built on: Mon Jul  4 11:25:51 2022 UTC
platform: debian-amd64
options:  bn(64,64) rc4(16x,int) des(int) blowfish(ptr) 
compiler: gcc -fPIC -pthread -m64 -Wa,--noexecstack -Wall -Wa,--noexecstack -g -O2 -fdebug-prefix-map=/build/openssl-wL7Fqk/openssl-1.1.1=. -fstack-protector-strong -Wformat -Werror=format-security -DOPENSSL_USE_NODELETE -DL_ENDIAN -DOPENSSL_PIC -DOPENSSL_CPUID_OBJ -DOPENSSL_IA32_SSE2 -DOPENSSL_BN_ASM_MONT -DOPENSSL_BN_ASM_MONT5 -DOPENSSL_BN_ASM_GF2m -DSHA1_ASM -DSHA256_ASM -DSHA512_ASM -DKECCAK1600_ASM -DRC4_ASM -DMD5_ASM -DAES_ASM -DVPAES_ASM -DBSAES_ASM -DGHASH_ASM -DECP_NISTZ256_ASM -DX25519_ASM -DPADLOCK_ASM -DPOLY1305_ASM -DNDEBUG -Wdate-time -D_FORTIFY_SOURCE=2
OPENSSLDIR: "/usr/lib/ssl"
ENGINESDIR: "/usr/lib/x86_64-linux-gnu/engines-1.1"
Seeding source: os-specific


Great! OpenSSL is present.
To conduct our tests we will need to store input and outputs in a text file.

Legend:

If cell starts with ! (e.g. !openssl), this refers to a Linux terminal command; else, it is a python code segment

## AES test

Test vectors for AES-128 from https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.197.pdf

We will use the CBC mode for AES-128; key and IV will be using the ones in the node-forge experiment, unless otherwise mentioned.

Official example vector (plaintext and key) from AES paper

AES-128 example vector:  
Plaintext: 00112233445566778899aabbccddeeff  
Key: 000102030405060708090a0b0c0d0e0f  
IV(randomly generated by node-forge):  e9ed0e320d9ade4e77dbf6ad170af249

In [ ]:
file = open('input.txt')
input = ''
for line in file:
  input=line
  print('input.txt contents:', input)
file.close()

input.txt contents: 00112233445566778899aabbccddeeff


In [ ]:
# reference from https://stackoverflow.com/questions/9889492/how-to-do-encryption-using-aes-in-openssl
# check last answer
# another good reference: https://stackoverflow.com/questions/16056135/how-to-use-openssl-to-encrypt-decrypt-files

!openssl enc -aes-128-cbc -in input.txt -out encrypted.txt -K 000102030405060708090a0b0c0d0e0f -iv e9ed0e320d9ade4e77dbf6ad170af249

In [ ]:
encrypt1 = ''
with open('encrypted.txt', 'rb') as f:
  encrypt1 = f.read().hex()
  print('encrypted.txt digest:', encrypt1)

encrypted.txt digest: f97ebf591ffb6c7319579932b8134549cdb18fa13b1e3f2b3e95185262648499ee6411e3591d32699b9941781991eb30


To ensure this command works we run the encrypt command a second time. We will see the same encrypted bytes in encrypted2.txt

In [ ]:
!openssl enc -aes-128-cbc -in input.txt -out encrypted2.txt -K 000102030405060708090a0b0c0d0e0f -iv e9ed0e320d9ade4e77dbf6ad170af249

In [ ]:
encrypt2 = ''
with open('encrypted2.txt', 'rb') as f:
  encrypt2 = f.read().hex()
  print('encrypted2.txt digest:', encrypt2)
  print()
  print('Are the contents of encrypted.txt and encrypted2.txt the same:', encrypt1==encrypt2)

encrypted2.txt digest: f97ebf591ffb6c7319579932b8134549cdb18fa13b1e3f2b3e95185262648499ee6411e3591d32699b9941781991eb30

Are the contents of encrypted.txt and encrypted2.txt the same: True


Now we use encrypted.txt's digest and decrypt to get our example plaintext!

In [ ]:
!openssl enc -aes-128-cbc -in encrypted.txt -out output.txt -d -K 000102030405060708090a0b0c0d0e0f -iv e9ed0e320d9ade4e77dbf6ad170af249

In [ ]:
output = ''
for line in open('output.txt'):
  output = line
  print('output.txt contents:', output)

print('input == output:',input==output)

output.txt contents: 00112233445566778899aabbccddeeff
input == output: True


Now we try OpenSSL using our own plaintext; key and IV are randomly generated by node-forge  
plaintext: Hello World  
Key: 17a4daa1a71e1fd0f19f55b6e37f607c  
IV: 688f4c8b18a35dc068a5459535fcfc4a  

In [ ]:
!openssl enc -aes-128-cbc -in input2.txt -out encrypted3.txt -K 17a4daa1a71e1fd0f19f55b6e37f607c -iv 688f4c8b18a35dc068a5459535fcfc4a

In [ ]:
!openssl enc -aes-128-cbc -in encrypted3.txt -d -K 17a4daa1a71e1fd0f19f55b6e37f607c -iv 688f4c8b18a35dc068a5459535fcfc4a | xxd -p

48656c6c6f20576f726c64


In [ ]:
with open('encrypted3.txt', 'rb') as f:
  print('encrypted3.txt digest:',f.read().hex())

encrypted3.txt digest: 27a339e1a595610bcc67508b2171a01f


In [ ]:
!openssl enc -aes-128-cbc -in encrypted3.txt -out output2.txt -d -K 17a4daa1a71e1fd0f19f55b6e37f607c -iv 688f4c8b18a35dc068a5459535fcfc4a

In [ ]:
input2 = 'Hello World'
output2 = ''
for line in open('output2.txt'):
  output2 = line
  print('output2.txt contents:', output2)

print('input == output:',input2==output2)

output2.txt contents: Hello World
input == output: True


This proves that OpenSSL AES implementation works the same way as node-forge AES.

## RSA test

### Verifying Node-forge RSA implementation via Python Code to do Wiener's Attack

This code calculates the convergents and their respective continuous fractions.

From node-forge, we generate a new 32-bit RSA private-public key pair for easy testing. Using details from text/rsa_32.txt:

public key:  {  
  n: BigInteger { data: [ 188750873, 8 ], t: 2, s: 0 },  
  e: BigInteger { data: [ 65537 ], t: 1, s: 0 },  
  encrypt: [Function (anonymous)],  
  verify: [Function (anonymous)]  
}

private key:  {  
  n: BigInteger { data: [ 188750873, 8 ], t: 2, s: 0 },  
  e: BigInteger { data: [ 65537 ], t: 1, s: 0 },  
  d: BigInteger { data: [ 214909973, 0 ], t: 1, s: 0 },  
  p: BigInteger { data: [ 62851 ], t: 1, s: 0 },  
  q: BigInteger { data: [ 37171 ], t: 1, s: 0 },  
  dP: BigInteger { data: [ 25823, 3419 ], t: 1, s: 0 },  
  dQ: BigInteger { data: [ 30203, 5781 ], t: 1, s: 0 },  
  qInv: BigInteger { data: [ 9621 ], t: 1, s: 0 },  
  decrypt: [Function (anonymous)],  
  sign: [Function (anonymous)]  
}

n = 2336234521  
e = 65537  
d = 214909973  
Wiener's condition: d < 73.28377868903891

From visual inspection, p and q are of equal sizes. We also see long d and short e, which means Wiener's Attack is likely to be unsuccessful. However let's assume we are the attacker and do not know size of d, so we try Wiener's Attack as follows.

In [13]:
e = 65537
n = 2336234521
print("Wiener's attack condition: d <", 1/3*n**0.25)

print("e/n =", e/n)
condition = e/n

print()
print("Using Extended Euclid's algorithm,")
q,r = 0,1
convergents = []
while r is not 0:
  q = e//n
  r = e%n
  print("{} = {}*{} + {}".format(e, q, n, r))
  e = n
  n = r
  convergents.append(q)

print()
print("convergents of e/n: ", convergents)
print("Start continued fractions...")
for i in range(1,len(convergents)):
  frac = 0.0
  for j in range(i,-1,-1):
    if j==i:
      frac = convergents[j]
    else:
      frac = convergents[j] + 1/frac
  print('convergent {}: {}'.format(i, frac))
  print('convergent < e/n == {}'.format(frac<condition))

Wiener's attack condition: d < 73.2837786890389
e/n = 2.8052406302064054e-05

Using Extended Euclid's algorithm,
65537 = 0*2336234521 + 65537
2336234521 = 35647*65537 + 37082
65537 = 1*37082 + 28455
37082 = 1*28455 + 8627
28455 = 3*8627 + 2574
8627 = 3*2574 + 905
2574 = 2*905 + 764
905 = 1*764 + 141
764 = 5*141 + 59
141 = 2*59 + 23
59 = 2*23 + 13
23 = 1*13 + 10
13 = 1*10 + 3
10 = 3*3 + 1
3 = 3*1 + 0

convergents of e/n:  [0, 35647, 1, 1, 3, 3, 2, 1, 5, 2, 2, 1, 1, 3, 3]
Start continued fractions...
convergent 1: 2.805285157236233e-05
convergent < e/n == False
convergent 2: 2.8052064631956913e-05
convergent < e/n == True
convergent 3: 2.8052458096640718e-05
convergent < e/n == False
convergent 4: 2.80524018867244e-05
convergent < e/n == True
convergent 5: 2.8052406774534268e-05
convergent < e/n == False
convergent 6: 2.8052406128974375e-05
convergent < e/n == True
convergent 7: 2.8052406324341183e-05
convergent < e/n == False
convergent 8: 2.8052406300427925e-05
convergent < e/n == True

Wiener's Attack condition, d<1/3 * n^0.25 = 73  
If Wiener's Attack condition works, d is a number less than 73

By calculating all the continued fractions of each convergent value, we observed that the closest convergent value is the last convergent where its value approaches the original e/n ratio.

However, we know the actual d>73, therefore Wiener's Attack wouldn't have worked (but attacker would not have known the size of d)

Unfortunately, we are unable to implement the private-public keypair from node-forge in OpenSSL, thus we show below that OpenSSL generates a long exponent d to prevent Wiener's Attack.

### OpenSSL implementation of RSA
reference: https://myprogrammingnotes.com/generate-modulus-rsa-using-openssl.html

OpenSSL allows us to generate n-bit keys with its details revealed to us (although not advisable to leave such files readable in practice).  
OpenSSL issue: OpenSSL only allows exponent e to be set as 3 or 65537

In [1]:
#line 1 generates private key of 512 bits
#line 2 exports the modulus, publicExponent = 65537 (0x10001), privateExponent, prime1, prime2, exponent1, exponent2, coefficient, private key

!openssl genrsa -out private.key 512
!openssl rsa -in private.key -text

Generating RSA private key, 512 bit long modulus (2 primes)
..+++++++++++++++++++++++++++
...........+++++++++++++++++++++++++++
e is 65537 (0x010001)
RSA Private-Key: (512 bit, 2 primes)
modulus:
    00:bd:b5:eb:19:17:9b:ef:37:6c:f2:03:57:bd:47:
    26:f0:c2:c0:20:5c:ef:84:6c:36:40:b0:5d:4b:0e:
    b4:63:51:7f:cf:00:64:c4:24:73:df:eb:bf:2d:42:
    fa:8f:8e:7a:38:dc:37:3a:f3:03:ff:6b:cc:fe:f5:
    07:65:2e:cb:2b
publicExponent: 65537 (0x10001)
privateExponent:
    00:82:b9:a0:cf:21:9d:aa:31:9f:39:05:64:a8:61:
    27:4c:30:67:03:e6:06:86:51:aa:f0:d8:96:f0:e9:
    a9:13:e6:1d:0a:ba:53:96:01:6c:3a:ff:ff:90:a5:
    d5:98:0e:2a:1d:72:ef:2b:fc:00:cf:9b:b5:9a:9d:
    73:28:01:5e:c1
prime1:
    00:e6:b6:fc:9a:81:37:8a:6d:b4:98:29:c6:cc:71:
    7b:dd:b6:40:ef:9f:91:cd:61:97:6f:fd:04:1d:12:
    eb:16:2f
prime2:
    00:d2:80:81:25:8c:7f:f8:06:13:cf:70:79:bd:c0:
    a1:73:7e:50:a7:2a:4d:f5:33:98:33:28:be:45:ab:
    9f:97:c5
exponent1:
    02:6f:a4:33:2d:72:1c:b0:13:dc:17:74:d5:eb:cc:
    d7:06:79:

Here we can see the details of 512-bit RSA private-public keypair.  
Our private exponent (d) in decimal: 4652668757374823945065059416577303681850322509010325553034580727783608823794850200913574502735000555189273390627662788348178638138017547063661675265824073

Thanks to its long length and the short length of public exponent e, we can safely say Wiener's Attack does not work on OpenSSL's implementation as well.

Note that trying to generate a key of size 256 bits and below (in powers of 2) leads to an error:

In [2]:
!openssl genrsa -out private2.key 256
!openssl rsa -in private2.key -text

Generating RSA private key, 256 bit long modulus (2 primes)
140238622021504:error:04081078:rsa routines:rsa_builtin_keygen:key size too small:../crypto/rsa/rsa_gen.c:78:
unable to load Private Key
140694638061440:error:0909006C:PEM routines:get_name:no start line:../crypto/pem/pem_lib.c:745:Expecting: ANY PRIVATE KEY
